In [0]:
!unzip -o "/content/drive/My Drive/plantdisease.zip"

Streaming output truncated to the last 5000 lines.
  inflating: plantdisease/valid/Tomato___Leaf_Mold/95a5ac3d-50d3-426c-83a6-3fd47fc8c58b___Crnl_L.Mold 6609(1).JPG  
  inflating: plantdisease/valid/Tomato___Leaf_Mold/95a5ac3d-50d3-426c-83a6-3fd47fc8c58b___Crnl_L.Mold 6609.JPG  
  inflating: plantdisease/valid/Tomato___Leaf_Mold/9622ac01-44eb-46b1-99cf-811d655ad6a8___Crnl_L.Mold 8801(1).JPG  
  inflating: plantdisease/valid/Tomato___Leaf_Mold/9622ac01-44eb-46b1-99cf-811d655ad6a8___Crnl_L.Mold 8801.JPG  
  inflating: plantdisease/valid/Tomato___Leaf_Mold/962f3d89-f439-46ee-b4bb-65409ee76fe1___Crnl_L.Mold 6914(1).JPG  
  inflating: plantdisease/valid/Tomato___Leaf_Mold/962f3d89-f439-46ee-b4bb-65409ee76fe1___Crnl_L.Mold 6914.JPG  
  inflating: plantdisease/valid/Tomato___Leaf_Mold/970a299e-18bd-4bf6-83af-f0950cfe0272___Crnl_L.Mold 8668(1).JPG  
  inflating: plantdisease/valid/Tomato___Leaf_Mold/970a299e-18bd-4bf6-83af-f0950cfe0272___Crnl_L.Mold 8668.JPG  
  inflating: plantdisease/valid/T

In [0]:
from keras import regularizers
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import Dropout
from keras.layers import GlobalAveragePooling2D
from keras.layers import BatchNormalization
from keras.layers import Activation,Dense
from keras.models import Sequential,load_model
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img

Using TensorFlow backend.


In [0]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [0]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        '/content/plantdisease/train',
        target_size=(150, 150),
        batch_size=32,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        '/content/plantdisease/valid',
        target_size=(150, 150),
        batch_size=32,
        class_mode='categorical')

Found 140585 images belonging to 38 classes.
Found 35144 images belonging to 38 classes.


In [0]:
from keras.applications.vgg16 import VGG16
base_vgg16 = VGG16(weights='imagenet', include_top = False, input_shape=(150,150,3))

58892288/58889256 [==============================] - 2s 0us/step


In [0]:
out = base_vgg16.output
out = GlobalAveragePooling2D()(out)
out = Dense(256, activation='relu')(out)
out = Dense(256, activation='relu')(out)
total_classes = 38
predictions = Dense(38, activation='softmax')(out)

In [0]:
model = Model(inputs=base_vgg16.input, outputs=predictions)

In [0]:
for layer in base_vgg16.layers:
    layer.trainable = False

In [0]:
model.compile(Adam(lr=.0001), loss='categorical_crossentropy', metrics=['accuracy']) 

In [0]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0   

In [0]:
model.fit_generator(
        train_generator,
        steps_per_epoch=2000,
        epochs=30,
        validation_data=validation_generator,
        validation_steps=500)

Epoch 1/30
2000/2000 [==============================] - 519s 259ms/step - loss: 1.9186 - accuracy: 0.5129 - val_loss: 0.8729 - val_accuracy: 0.6872
Epoch 2/30
2000/2000 [==============================] - 480s 240ms/step - loss: 0.9468 - accuracy: 0.7187 - val_loss: 0.5071 - val_accuracy: 0.7688
Epoch 3/30
2000/2000 [==============================] - 453s 226ms/step - loss: 0.7463 - accuracy: 0.7713 - val_loss: 0.8325 - val_accuracy: 0.7957
Epoch 4/30
2000/2000 [==============================] - 447s 223ms/step - loss: 0.6508 - accuracy: 0.7968 - val_loss: 0.5883 - val_accuracy: 0.8138
Epoch 5/30
2000/2000 [==============================] - 450s 225ms/step - loss: 0.5876 - accuracy: 0.8119 - val_loss: 0.3833 - val_accuracy: 0.8326
Epoch 6/30
2000/2000 [==============================] - 450s 225ms/step - loss: 0.5378 - accuracy: 0.8287 - val_loss: 0.6877 - val_accuracy: 0.8345
Epoch 7/30
2000/2000 [==============================] - 451s 226ms/step - loss: 0.5123 - accuracy: 0.8352 - val_

In [0]:
model.save_weights("model.h5")

In [0]:
test_generator = test_datagen.flow_from_directory(
    directory=r"/content/plantdisease/test",
    target_size=(150, 150),
    color_mode="rgb",
    batch_size=1,
    class_mode=None,
    shuffle=False,
    seed=42
)

Found 33 images belonging to 1 classes.


In [0]:
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
test_generator.reset()
pred=model.predict_generator(test_generator,
steps=STEP_SIZE_TEST,
verbose=1)

33/33 [==============================] - 1s 25ms/step


In [0]:
predicted_class_indices=np.argmax(pred,axis=1)

In [0]:
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

In [0]:
import pandas as pd
filenames=test_generator.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})
results.to_csv("results.csv",index=False)